### Save yamnet embeddings

In [23]:
# Imports.
import params as yamnet_params
import yamnet as yamnet_model

import tensorflow as tf
import tensorflow_io as tfio
import numpy as np
import glob
import time
import os
import tqdm
import librosa
from pathlib import Path

### Paths

In [35]:
data_path = '/Users/fabianseipel/aic/data/rfcx-species-audio-detection/'

### Get and save embeddings

In [41]:
# The graph is designed for a sampling rate of 16 kHz, but higher rates should work too.
# We also generate scores at a 10 Hz frame rate.
params = yamnet_params.Params(sample_rate=48000, patch_hop_seconds=0.1)
        
# Set up the YAMNet model.
class_names = yamnet_model.class_names('yamnet_class_map.csv')
yamnet = yamnet_model.yamnet_frames_model(params)
yamnet.load_weights('yamnet.h5')

# save embedding per 1s frame (predict middle 0.96s)
def save_yamnet_embeddings(f):
    # construct output path
    savepath = './embeddings/' + Path(f).parts[-2] + '/' + str(Path(f).stem) + '.npy' 
    
    # save embeddings if not exist already
    if not os.path.exists(savepath):      
        audio = tfio.audio.AudioIOTensor(f)
        audio = tf.squeeze(audio[:], axis=[-1])
        frames = librosa.util.frame(audio.numpy(), frame_length=48000, hop_length=48000)
        n_frames = frames.shape[1]
        embeddings = []

        for n in range(n_frames):
            # get mid 0.96 seconds
            audio_frame = frames[int(0.02*48000):-int(0.02*48000), n]
            _, emb, _ = yamnet(audio_frame)
            embeddings.append(emb.numpy())

        # save embeddings
        emb_array = np.squeeze(np.array(embeddings))
        np.save(savepath, emb_array)

# script
test_files = glob.glob(data_path + 'test/*.flac')
train_files = glob.glob(data_path + 'train/*.flac')
files = test_files + train_files

# make folders
os.makedirs('./embeddings/', exist_ok=True)
os.makedirs('./embeddings/test', exist_ok=True)
os.makedirs('./embeddings/train', exist_ok=True)

'''
for f in tqdm.tqdm(files):
    save_yamnet_embeddings(f)
'''

print()